## Model Representation
<img align="left" src="../images/C1_W1_L3_S1_Lecture_b.png"    style=" width:96%; height:250px; padding: 10px;  " />
<img align="left" src="../images/C1_W1_L3_S1_trainingdata.png"    style=" width:47%; height:200px; padding: 10px;  " />
<img align="left" src="../images/C1_W1_L3_S1_model.png"    style=" width:47%; height:200px; padding: 10px;  " />
<img align="left" src="../images/C1_W1_L3_S2_Lecture_b.png"    style=" width:96%; height:250px; padding: 10px;  " />
<img align="left" src="../images/C1_W1_Lab02_GoalOfRegression.PNG"    style=" width:96%; height:200px; padding: 10px;  " />
<img align="left" src="../images/C1_W1_L4_S1_Lecture_GD.png"    style=" width:96%; height:250px; padding: 10px;  " />
<img align="left" src="../images/C1_W1_Lab03_lecture_slopes.PNG"    style=" width:47%; height:200px; padding: 10px;  " />
<img align="left" src="../images/C1_W1_Lab03_lecture_learningrate.PNG"    style=" width:47%; height:200px; padding: 10px;  " />
<img align="left" src="../images/C1_W1_Lab03_alpha_too_big.PNG"    style=" width:96%; height:250px; padding: 10px;  " />

In [6]:
import math, copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('./deeplearning.mplstyle')
from lab_utils_uni import plt_house_x, plt_contour_wgrad, plt_divergence, plt_gradients

In [57]:
df = pd.read_csv('./data.csv')
df.columns = df.columns.str.lower().str.replace(' ','_')

list_of_head = list(df.dtypes[df.dtypes != 'object'].index)
# list_of_head
df['engine_hp'].fillna(round(df['engine_hp'].mean(),2),inplace=True)
x_feature = df['engine_hp'].iloc[:101]
y_target = df['msrp'].iloc[:101]
# df['msrp'].isnull().sum()
# for col in list_of_head:
#     df[col] = df[col].str.lower().str.replace(' ', '_')
# for col in df.columns:
#     print(df[col].unique()[:5])
#     print(df[col].nunique())
#     print()